In [1]:
import pandas as pd
import numpy as np
import re
import time

In [2]:
lines = open('data/cornell_movie_dialogs_corpus/movie_lines.txt', 'r', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('data/cornell_movie_dialogs_corpus/movie_conversations.txt', 'r', encoding='utf-8', errors='ignore').read().split('\n')

In [3]:
lines[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

In [4]:
conv_lines[:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

In [5]:
id2line = {}
for line in lines:
    if len(line) > 0:
        lineID, characterID, movieID, character, text = line.split(" +++$+++ ")
        id2line[lineID] = text

In [6]:
convs = [ ]
for _conv in conv_lines:
    if len(_conv) > 0:
        character1, character2, movieID, conv = _conv.split(" +++$+++ ")
        convs.append(eval(conv))

In [7]:
convs[:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

In [8]:
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])

In [9]:
limit = 0
for i in range(limit, limit+5):
    print(questions[i])
    print(answers[i])
    print()

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?
Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.



In [10]:
assert len(questions)==len(answers)

In [11]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,*]", "", text)
    
    return text

In [12]:
questions = list(map(clean_text, questions))

In [13]:
answers = list(map(clean_text, answers))

In [14]:
import torch

In [85]:
class dataset(torch.utils.data.Dataset):
    def __init__(self, questions, answers, l=[2, 30], loading=False):
        """
        questions: X seq list
        answers: Y seq list
        l: length range [min, max]
        loading: loading word2index.csv or generate new one
        """
        super(dataset, self)
        questions = np.array(questions)
        answers = np.array(answers)
        self.min_length = l[0]
        self.max_length = l[1]
        questions, answers = self.length_filter(questions, answers)
        questions, answers = self.length_filter(answers, questions)
        self.word2index = {}
        self.index2word = {}
        self.load_vocab(questions, answers, loading, 5)
        
        self.questions, self.answers = self.sentences2tensors(questions, answers)
#         self.questions = np.core.defchararray.add(questions, ' <EOS>')
#         self.answers = np.core.defchararray.add(answers, ' <EOS>')
        
    def length_filter(self, data1, data2):
        """filter length based on data1"""
        d1_len = np.array(list(map(lambda x: len(x.split()), data1)))
        d1_idx = np.logical_and(d1_len<=self.max_length, d1_len>=self.min_length)
        return data1[d1_idx], data2[d1_idx]
    
    def load_vocab(self, questions, answers, loading, threshold=5):
        """generate vocabulary"""
        import csv, os, sys
        if not loading:
            words_count = {}
            for question in questions:
                for word in question.split():
                    if word not in words_count:
                        words_count[word] = 1
                    else: words_count[word] += 1
            for ans in answers:
                for word in ans.split():
                    if word not in words_count:
                        words_count[word] = 1
                    else: words_count[word] += 1
            count = 0
            for k,v in words_count.items():
                if v >= threshold:
                    count += 1
            print("Size of total vocab:", len(words_count))
            print("Size of vocab will be used:", count)
            word2index = {'<PAD>':0,'<EOS>':1,'<UNK>':2,'<GO>':3}
            word_idx = 4
            for word, count in words_count.items():
                if count >= threshold:
                    word2index[word]=word_idx
                    word_idx += 1

            file_w2i = open("word2index.csv", "w")        
            writer = csv.writer(file_w2i)
            for key, value in word2index.items():
                writer.writerow([key, value])
            file_w2i.close()
        elif loading and not os.path.exists('word2index.csv'):
            sys.stderr.write(""*40)
            sys.stderr.write("\n\nNo Such File Found\n")
            sys.stderr.write("\nMust be .csv file\nFile must under current directory\nFile must named word2index.csv")
        else:
            print("Loading word2index.csv")
            with open("word2index.csv", "r") as csvfile:
                reader = csv.reader(csvfile, delimiter=',')
                word2index = {word: eval(idx) for (word, idx) in reader}
            print("Loading complete")
            
        self.word2index = word2index
        self.index2word = {idx : w for w, idx in word2index.items()}
        print("Size of vocab:", len(word2index))
        
    
    def sentences2tensors(self, questions, answers):
        questions = np.core.defchararray.add(questions, ' <EOS>').tolist()
        answers = np.core.defchararray.add(answers, ' <EOS>').tolist()
        
        def sent2tensor(sentence):
            words = sentence.split()
            seq = np.zeros(self.max_length+1)
            for i, word in enumerate(words):
                if word not in self.word2index:
                    seq[i] = self.word2index['<UNK>']
                else:
                    seq[i] = self.word2index[word]
            return seq
        
        q_tensors = torch.tensor(list(map(sent2tensor, questions))).long()
        a_tensors = torch.tensor(list(map(sent2tensor, answers))).long()
        print("Total number of questions", q_tensors.shape)
        print("Total number of answers", a_tensors.shape)
        
        num_not_padding = (q_tensors!=0).sum() + (a_tensors!=0).sum() - q_tensors.shape[0]*2
        num_unk = (q_tensors==2).sum() + (a_tensors==2).sum()
        print("Percent of words that are <UNK>: {:.02f}%".format(num_unk.float()/num_not_padding.float()*100))

        
        return q_tensors, a_tensors
            
    
    def __getitem__(self, i):
        X = self.questions[i, :]
        y = self.answers[i, :]
        return X, y
    
        
    def __len__(self):
        return self.questions.shape[0]
        

In [86]:
data = dataset(questions, answers, loading=True)

Loading word2index.csv
Loading complete
Size of vocab: 16877
Total number of questions torch.Size([165571, 31])
Total number of answers torch.Size([165571, 31])
Percent of words that are <UNK>: 2.18%


In [88]:
torch.utils.data.DataLoader(data)

(tensor([14, 24, 25, 26,  7, 27, 28, 29, 30, 31, 32, 33,  1,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 tensor([16, 17, 18, 19, 20, 19, 21, 22, 23,  1,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]))